In [ ]:
import datetime
import json
import time

#  动量选股
from jqdata import *

import fun

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

res_stocks = []

In [ ]:
# 去除上市不满半年的股票
all_stocks = get_all_securities()
all_stocks = all_stocks[all_stocks['start_date'] < (datetime.date.today() - datetime.timedelta(days=180))]

# 筛选市值大于 100 亿的
df_val = get_fundamentals(query(valuation.code, valuation.market_cap).filter(valuation.market_cap >= 100).order_by(
            # 按市值降序排列
            valuation.market_cap.desc()
        ), date=None)
mc_stocks = list(df_val['code'])
print('市值大于 50亿 股票数量：', len(mc_stocks))

# 收入净利润增长的公司
# df_ind = get_fundamentals(query(
#     indicator.code, indicator.pubDate, indicator.statDate, indicator.eps, indicator.inc_revenue_year_on_year,
#     indicator.inc_operation_profit_year_on_year, indicator.inc_net_profit_year_on_year)
#                  .filter(indicator.inc_net_profit_year_on_year > 0)
#                  .order_by(indicator.inc_net_profit_year_on_year.desc())
#                  , date=None)
# ind_stocks = list(df_ind['code'])
# print('净利润增长的股票数量：', len(ind_stocks))

df_stocks = set(mc_stocks) # & set(ind_stocks)

run_codes = []
for code in list(all_stocks.index):
    if code in df_stocks:
        run_codes.append(code)

print('最终筛选运行股票数量：', len(run_codes))

# 获取股票行情信息
stock_bars = get_bars(run_codes, count=20, unit='1d',fields=['date', 'open','high','low','close'], include_now=True, end_dt=None, df=True)
# 计算股票 20 日涨跌幅
stock_20rate_rank = []
for code in run_codes:
    try:
        bar = stock_bars.loc[(code, slice(None)),:]
        if len(bar) == 20:
            rate = (bar['close'].iloc[-1] - bar['close'].iloc[0]) / bar['close'].iloc[0]
            stock_20rate_rank.append({'code': code, 'rate': rate})
    except KeyError:
        pass
# 涨跌幅排序
stock_20rate_rank.sort(key=lambda r:r['rate'], reverse=True)
# 选取排名前 200 选手
stocks = stock_20rate_rank[0:200]
rank_stocks = list(map(lambda s:s['code'], stocks))
# 筛选基金持股或北向持股的股票
stocks_all = []
for code in rank_stocks:
    if fun.stock_jj_bx_share_ok(code):
        stocks_all.append(code)

print('基金筛选后剩余股票数量：', len(stocks_all))
# 获取股票行情信息
stocks_industry = get_industry(security=stocks_all)
stocks_concept = get_concept(security=stocks_all, date=datetime.datetime.now())
# 按照行业概念汇总股票列表
concept_stocks = {}
industry_stocks = {}
for code in stocks_industry:
    hy = stocks_industry[code]
    if 'jq_l2' not in hy:
        continue
    hy_code = hy['jq_l2']['industry_code']
    hy_name = hy['jq_l2']['industry_name']
    if hy_code not in industry_stocks:
        industry_stocks[hy_code] = {'hy_code': hy_code, 'hy_name': hy_name, 'cf_stocks': [code]}
        industry_stocks[hy_code]['all_stocks'] = get_industry_stocks(hy_code)
    else:
        industry_stocks[hy_code]['cf_stocks'].append(code)
# 计算行业动量分值
for hy in industry_stocks:
    # 成分股票数量 * ( 成分股票数量 / 行业股票数量 )
    industry_stocks[hy]['score'] = len(industry_stocks[hy]['cf_stocks']) * (len(industry_stocks[hy]['cf_stocks']) / len(industry_stocks[hy]['all_stocks']))

# 动量值排序
industry_ranks = sorted(industry_stocks.items(), key=lambda d:d[1]['score'], reverse=True)

for code in stocks_concept:
    co = stocks_concept[code]
    for _info in co['jq_concept']:
        co_code = _info['concept_code']
        co_name = _info['concept_name']
        if co_name in ['深股通', '转融券标的', '融资融券', '沪股通', 'MSCI', '国家大基金持股']:
            continue
        if co_code not in concept_stocks:
            concept_stocks[co_code] = {'co_code': co_code, 'co_name': co_name, 'cf_stocks': [code]}
            concept_stocks[co_code]['all_stocks'] = get_concept_stocks(co_code)
        else:
            concept_stocks[co_code]['cf_stocks'].append(code)

# 计算概念动量分值
for co in concept_stocks:
    # 成分股票数量 * ( 成分股票数量 / 行业股票数量 )
    concept_stocks[co]['score'] = len(concept_stocks[co]['cf_stocks']) * (len(concept_stocks[co]['cf_stocks']) / len(concept_stocks[co]['all_stocks']))

# 动量值排序
concept_ranks = sorted(concept_stocks.items(), key=lambda d:d[1]['score'], reverse=True)

# 获取成分股票名称
for hy in industry_ranks:
    stock_names = []
    for _c in hy[1]['cf_stocks']:
        stock = get_security_info(_c)
        stock_names.append(stock.display_name)
    hy[1]['cf_names'] = stock_names

# 获取概念股票名称
for co in concept_ranks:
    stock_names = []
    for _c in co[1]['cf_stocks']:
        stock = get_security_info(_c)
        stock_names.append(stock.display_name)
    co[1]['cf_names'] = stock_names

In [ ]:
# 输出动量排行榜单
print("行业名称/分数/成分/总数")
for hy in industry_ranks:
    print("%s/%s/%s/%s" % (hy[1]['hy_name'], hy[1]['score'], len(hy[1]['cf_stocks']), len(hy[1]['all_stocks'])))

In [ ]:
print(json.dumps(industry_ranks, ensure_ascii=False))

In [ ]:
# 输出动量排行榜单
print("概念名称/分数/成分/总数")
for co in concept_ranks:
    print("%s/%s/%s/%s" % (co[1]['co_name'], co[1]['score'], len(co[1]['cf_stocks']), len(co[1]['all_stocks'])))

In [ ]:
print(json.dumps(concept_ranks, ensure_ascii=False))

In [ ]:
print('行业热门股票')
# 获取分值 大于 5 的股票列表
high_stocks = []
rank_stocks = []

for hy in industry_ranks[0:10]:
    high_stocks += hy[1]['cf_stocks'][0:3]
    rank_stocks += hy[1]['cf_stocks']
print('最热股票数量：', len(high_stocks))
print('排行股票数量：', len(rank_stocks))

In [ ]:
print('概念热门股票')
# 获取分值 大于 5 的股票列表
co_high_stocks = []
co_rank_stocks = []
for co in concept_ranks[0:10]:
    co_high_stocks += co[1]['cf_stocks'][0:3]
    co_rank_stocks += co[1]['cf_stocks']
print('最热股票数量：', len(co_high_stocks))
print('排行股票数量：', len(co_rank_stocks))


print(' * ' * 20)
double_high_stocks = set(high_stocks) & set(co_high_stocks)
double_rank_stocks = set(rank_stocks) & set(co_rank_stocks)
print('热门重合股票：', fun.stock_names(double_high_stocks))
print('排行重合股票：', fun.stock_names(double_rank_stocks))
print(' * ' * 20)

high_stocks += co_high_stocks
rank_stocks += co_rank_stocks
high_stocks = list(set(high_stocks))
rank_stocks = list(set(rank_stocks))
print('最热股票数量：', len(high_stocks))
print('排行股票数量：', len(rank_stocks))

In [ ]:
# 输出符合条件的股票列表
_run_codes = []
for code in run_codes:
    _run_codes.append(fun.reformat_code(code))
print(json.dumps(_run_codes, ensure_ascii=False))